In [1]:
import numpy as np
import torch
from torch import nn
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print("Running on",device)

Running on cuda:0


In [2]:
dat = np.genfromtxt("../input/mnist-in-csv/mnist_train.csv", delimiter = ",", skip_header = 1)

In [3]:
X, y = dat[:,1:]/255.0, dat[:,0]

X = torch.tensor(X, requires_grad=True, dtype = torch.double).view(-1,28,28).unsqueeze(1).to(device)


y = torch.tensor(y, requires_grad=False, dtype = int).to(device)

In [4]:
if device == "cpu":
    i = 5
    plt.imshow(X[i].detach().numpy()[0])
    plt.show()
    
    plt.imshow(X[i+1].detach().numpy()[0])
    plt.show()

    plt.imshow(X[i].detach().numpy()[0] + X[i+1].detach().numpy()[0])
    plt.title(str(y[i].numpy() + y[i+1].numpy()))
    plt.show()


In [5]:
# def onehot(y):
#     out = torch.zeros(y.shape[0],10)
#     for i in range(y.shape[0]):
#         out[i][y[i]] = 1
#     return out

def batch_loader(X,y, batch_size = 6):
    for i in range(y.shape[0]//batch_size):
        X_out = X[i*batch_size:(i+1)*batch_size]
        

        y_out = y[i*batch_size:(i+1)*batch_size]

        # y_out = onehot(y_out)
        yield X_out, y_out
        
        
def batch_loader_roll(X,y, batch_size = 6):
    for i in range(y.shape[0]//batch_size):
        XX = X[i*batch_size:((i+1)*batch_size-1)]
        X_out = XX + torch.roll(XX, 1, 0)
        
        yy = y[i*batch_size:((i+1)*batch_size -1)]
        y_out = yy + torch.roll(yy, 1, 0)

        # y_out = onehot(y_out)
        yield X_out, y_out

In [6]:
class CNN(nn.Module):
    def __init__(self,SIZE = 100, pretrain = -1):
        super (CNN, self).__init__()

        features = 20
        if pretrain == 0:
            features = 4
        if pretrain == 1:
            features = 2
        if pretrain == 2:
            features = 2
        if pretrain == 3:
            features = 2

        self.SIZE = SIZE

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=(2, 2))
        self.activation1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.conv2 = nn.Conv2d(in_channels=64, out_channels=SIZE, kernel_size=(5, 5))
        self.activation2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        
        self.conv3 = nn.Conv2d(in_channels=SIZE, out_channels=SIZE*2*2, kernel_size=(5, 5), padding = 2)
        self.activation3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))

        self.linear1 = nn.Linear(in_features=SIZE*4*4, out_features=SIZE*4*4, )
        self.activation_linear = nn.ReLU()
        
        self.linear2 = nn.Linear(in_features=SIZE*4*4, out_features=SIZE*4*4//2, )
        self.activation_linear2 = nn.ReLU()

        self.output = nn.Linear(in_features=SIZE*4*4//2, out_features = features)
        # self.final_activation = nn.Softmax(dim=0)
        self.final_activation = lambda x: x

    def forward(self, x):
        x = self.activation1(self.conv1(x))
        x = self.maxpool1(x)

        x = self.activation2(self.conv2(x))
        x = self.maxpool2(x)
        
        x = self.activation3(self.conv3(x))
        x = self.maxpool3(x)
        
        
        x = x.view(-1,self.SIZE*4*4)
        x = self.activation_linear(self.linear1(x))
        x = self.activation_linear2(self.linear2(x))
        x = self.output(x)

        out = self.final_activation(x)

        return out

    def pretrain(self, x, _type = 0):
        l = x.shape[0]
        _x = x.clone().detach()

        if _type == 0:
            n = np.random.randint(0,4, l)

            for i in range(l):
                _x[i] = torch.rot90(_x[i], int(n[i]), [1,2])
                
            y = n
            
        if _type == 1:
            c = np.random.random(l)<0.5

            # wow
            b = torch.tensor(c.astype(int)).unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(31,-1,28,28).to(device)

            noise = torch.randn_like(_x)*0.1 * b
            _x = _x + noise
            
            y = c
            
        if _type == 2:
            c = np.random.random(l)<0.5

            noise = torch.roll(_x, 1, 0) * torch.tensor(c.astype(int)).unsqueeze(1).unsqueeze(2).unsqueeze(3).expand(31,-1,28,28).to(device)
            _x = _x + noise
            y = c      
        
        return _x, y





In [7]:
loss_fn = nn.CrossEntropyLoss()

batch_size = 32

def pretrain(model, n_epochs, pretrain_mode ):
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.0001)

    model.train()
    print("Pre-training")

    for i in range(n_epochs):

        c = 0
        epoch_loss = 0
        epoch_acc = 0
        
        Loader = batch_loader_roll(X,y, batch_size)
        pbar = tqdm(Loader, total = y.shape[0]//batch_size)
        losses = []
        for dat, _ in pbar:
            dat, label = model.pretrain(dat, pretrain_mode)
            label = torch.tensor(label, dtype = torch.long).to(device)
            
            pred = model(dat.float())
            loss = loss_fn(pred, label)

            loss.backward()
            optimizer.step()
            losses.append(loss.cpu().detach().numpy())
            epoch_loss += loss

            corr = (label == pred.argmax(axis = 1))

            epoch_acc += corr.float().mean()
            c += 1

            if c%10 == 0:
                pbar.set_description(f"loss {epoch_loss/c:.3} - acc {epoch_acc/c:.3}")
            # if c%100 == 0:
                # print(pred[:5],label[:5],pred[:5].argmax(axis = 1) == label[:5])
        print(f"Epoch {i+1}/{n_epochs} - loss {epoch_loss/c:.3} - acc {epoch_acc/c:.3}")
    return losses
            

def fine_tune(model, n_epochs):
    # for param in model.parameters():
    #         param.requires_grad = False
    print("Fine-tuning")
    model.output = nn.Linear(in_features = model.SIZE*4*4//2, out_features = 20)
    optimizer = torch.optim.Adam(params = model.parameters(), lr = 0.00001)
    model.to(device)
    losses = []
    accuracies = []
    for i in range(n_epochs):
        Loader = batch_loader_roll(X,y, batch_size)

        c = 0
        epoch_loss = 0
        epoch_acc = 0

        pbar = tqdm(Loader, total = y.shape[0]//batch_size)
        for dat, label in pbar:
            dat = torch.tensor(dat).to(device)
            label = torch.tensor(label).to(device)

            pred = model(dat.float())
            loss = loss_fn(pred, label)

            loss.backward()
            optimizer.step()
            losses.append(loss.cpu().detach().numpy())
            epoch_loss += loss
            acc = (label == pred.argmax(axis = 1)).float().mean()
            epoch_acc += acc
            accuracies.append(acc.cpu().detach().numpy())
            c += 1

            if c%10 == 0:
                pbar.set_description(f"loss {epoch_loss/c:.3} - acc {epoch_acc/c:.3}")
            # if c%100 == 0:
                # print(pred[:5],label[:5],pred[:5].argmax(axis = 1) == label[:5])
        print(f"Epoch {i+1}/{n_epochs} - loss {epoch_loss/c:.3} - acc {epoch_acc/c:.3}")
    return losses, accuracies

In [8]:
SIZE = 200
model = CNN(SIZE, pretrain = -1)
model.to(device)
losses, accuracies = fine_tune(model, 10)


Fine-tuning


  0%|          | 0/1875 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 1/10 - loss 2.84 - acc 0.0929


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8/10 - loss 2.8 - acc 0.0988


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9/10 - loss 2.8 - acc 0.0974


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 10/10 - loss 2.8 - acc 0.0972


In [9]:
def triple_threat():
    SIZE = 100
    model = CNN(SIZE, pretrain = 0)
    pretrain_losseses = []
    for i in range(3):
        mode = i%3
        features = 2
        if mode == 0:
            features = 4

        model.output = nn.Linear(in_features=SIZE*4*4//2, out_features = features)
        model.to(device)

        pretrain_losses = pretrain(model, 1, pretrain_mode = mode)
        
        pretrain_losseses.append(pretrain_losses)
    return pretrain_losseses

In [10]:
for pretrain_mode in [-1,0,1,2,3]:
    print(" ")
    for i in range(2):
        print("Pretrain mode is",pretrain_mode)
        model = CNN(100, pretrain = pretrain_mode)

        model.to(device)
        
        pretrain_losses = []
        if pretrain_mode != -1:
            if pretrain_mode == 3:
                pretrain_losses = triple_threat()
            else:
                pretrain_losses = pretrain(model, 1, pretrain_mode = pretrain_mode)
        
        losses, accuracies = fine_tune(model, 10)

        run = {"pretrain mode": pretrain_mode,"pretrain losses" : pretrain_losses, "fine-tune losses" : losses, "fine-tune accuracies": accuracies}
        np.save("pretain_data_"+str(pretrain_mode)+"_run" + str(i) +".npy", run, allow_pickle = True)
        print("Data saved!")

 
Pretrain mode is -1
Fine-tuning


  0%|          | 0/1875 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:63: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 1/10 - loss 2.84 - acc 0.0932


  0%|          | 0/1875 [00:00<?, ?it/s]

...

Epoch 8/10 - loss 1.4 - acc 0.544


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 9/10 - loss 1.28 - acc 0.592


  0%|          | 0/1875 [00:00<?, ?it/s]

Epoch 10/10 - loss 1.19 - acc 0.622
Data saved!
